# YOLOv8 Test Notebook
We will use this notebook as a method for testing and running the YOLOv8 architecture

In [2]:
from ultralytics import YOLO

# Install and load the yolo v8 model
model = YOLO("yolov8n.pt")

In [ ]:
print(model)